<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Web_Search_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -q llama-index==0.10.5 openai==1.12.0 tiktoken==0.6.0 llama-index-tools-google==0.1.3 newspaper3k==0.2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [2]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
GOOGLE_SEARCH_KEY = "[GOOGLE_SEARCH_KEY]"
GOOGLE_SEARCH_ENGINE = "[GOOGLE_SEARCH_ENGINE]"

# Using Agents/Tools

## Define Google Search Tool

In [3]:
from llama_index.tools.google import GoogleSearchToolSpec

tool_spec = GoogleSearchToolSpec(key=GOOGLE_SEARCH_KEY, engine=GOOGLE_SEARCH_ENGINE)

In [55]:
# Import and initialize our tool spec
from llama_index.core.tools.tool_spec.load_and_search import LoadAndSearchToolSpec

# Wrap the google search tool to create an index on top of the returned Google search
wrapped_tool = LoadAndSearchToolSpec.from_defaults(
    tool_spec.to_tool_list()[0],
).to_tool_list()

## Create the Agent

In [56]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(wrapped_tool, verbose=False)

In [ ]:
res = agent.chat("How many parameters LLaMA2 model has?")

In [ ]:
res.response

'The LLaMA2 model has parameters available in three different sizes: 7 billion, 13 billion, and 70 billion.'

In [ ]:
res.sources

[ToolOutput(content='Content loaded! You can now search the information using read_google_search', tool_name='google_search', raw_input={'args': (), 'kwargs': {'query': 'parameters of LLaMA2 model'}}, raw_output='Content loaded! You can now search the information using read_google_search', is_error=False),
 ToolOutput(content='Answer: The parameters of the LLaMA2 model are available in three different sizes: 7 billion, 13 billion, and 70 billion.', tool_name='read_google_search', raw_input={'args': (), 'kwargs': {'query': 'parameters of LLaMA2 model'}}, raw_output='Answer: The parameters of the LLaMA2 model are available in three different sizes: 7 billion, 13 billion, and 70 billion.', is_error=False)]

# Using Tools w/ VectorStoreIndex

A limitation of the current agent/tool in LlamaIndex is that it **relies solely on the page description from the retrieved pages** to answer questions. This approach will miss answers that are not visible in the page's description tag. To address this, a possible workaround is to fetch the page results, extract the page content using the newspaper3k library, and then create an index based on the downloaded content. Also, the previous method stacks all retrieved items from the search engine into a single document, making it **difficult to pinpoint the exact source** of the response. However, the following method will enable us to present the sources easily.

## Define Google Search Tool

In [ ]:
from llama_index.tools.google import GoogleSearchToolSpec

tool_spec = GoogleSearchToolSpec(key=GOOGLE_SEARCH_KEY, engine=GOOGLE_SEARCH_ENGINE)

In [44]:
search_results = tool_spec.google_search("LLaMA2 model details")

In [46]:
import json

search_results = json.loads( search_results[0].text )

## Read Each URL Contents

In [47]:
import newspaper
pages_content = []

for item in search_results['items']:

    try:
        article = newspaper.Article( item['link'] )
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append({ "url": item['link'], "text": article.text, "title": item['title'] })
    except:
        continue

print(len(pages_content))

8


## Create the Index

In [48]:
from llama_index.core import Document

# Convert the texts to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row["text"], metadata={"title": row["title"], "url": row["url"]}) for row in pages_content]

In [49]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

# Build index / generate embeddings using OpenAI.
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[SentenceSplitter(chunk_size=512, chunk_overlap=64)],
)

In [50]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine()

## Query

In [52]:
response = query_engine.query(
    "How many parameters LLaMA2 model has?"
)
print(response)

LLaMA2 model has sizes ranging from 7 to 70 billion parameters.


In [53]:
response = query_engine.query(
    "How many parameters LLaMA2 model has? list exact sizes."
)
print(response)

The LLaMA2 model comes in several sizes with different numbers of parameters:
- LLaMA2 7B
- LLaMA2 13B
- LLaMA2 33B
- LLaMA2 65B


In [62]:
# Show the retrieved nodes
for src in response.source_nodes:
  print("Title\t", src.metadata['title'])
  print("Source\t", src.metadata['url'])
  print("Score\t", src.score)
  print("-_"*20)

Title	 Introducing LLaMA: A foundational, 65-billion-parameter language ...
Source	 https://ai.meta.com/blog/large-language-model-llama-meta-ai/
Score	 0.8124383491026671
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Title	 Llama 2 follow-up: too much RLHF, GPU sizing, technical details
Source	 https://www.interconnects.ai/p/llama-2-part-2
Score	 0.8046542892214631
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
